In [41]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import time

plt.rcParams.update({'font.size': 18})

In [42]:
h5_file = "test_file.h5"

In [43]:
print ("Reading",h5_file)
hf = h5py.File(h5_file, 'r')
print (np.where(hf["data/obssource"][()]==1)[0])

# print (bytes(hf["data/obssource"][0]).decode("ascii"))
# print (hf["data/obssource"][()].shape)
hf.close()

Reading test_file.h5
[ 1  4  7 10]
